#### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
loader=WebBaseLoader("https://docs.langchain.com/langsmith/evaluation-quickstart")
loader

In [5]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/evaluation-quickstart', 'title': 'Evaluation quickstart - Docs by LangChain', 'language': 'en'}, page_content='Evaluation quickstart - Docs by LangChainOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KGitHubForumForumSearch...NavigationQuickstartsEvaluation quickstartGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGitHubForumOn this pageLangSmith SDK1. Install dependencies2. Create a LangSmith API key3. Set up environment variables4. 

In [6]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [7]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/evaluation-quickstart', 'title': 'Evaluation quickstart - Docs by LangChain', 'language': 'en'}, page_content='Evaluation quickstart - Docs by LangChainOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQOur new LangChain Academy course on Deep Agents is now live! Enroll for free.Docs by LangChain home pagePythonSearch...⌘KGitHubForumForumSearch...NavigationQuickstartsEvaluation quickstartGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGitHubForumOn this pageLangSmith SDK1. Install dependencies2. Create a LangSmith API key3. Set up environment variables4. 

In [8]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

c:\Users\DeLL\Documents\GitHub\OpenAI-and-Ollama\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [10]:
vectorstoredb

In [11]:
## Query From a vector db
query="A target function that defines what you’re evaluating."
result=vectorstoredb.similarity_search(query)
result[0].page_content

'A dataset with test inputs and optionally expected outputs.\nA target function that defines what you’re evaluating. For example, this may be one LLM call that includes the new prompt you are testing, a part of your application or your end to end application.\nEvaluators that score your target function’s outputs.\n\nThis quick start guides you through running a simple evaluation to test the correctness of LLM responses with the LangSmith SDK or UI.\nThis guide uses prebuilt LLM-as-judge evaluators from the open-source openevals package. OpenEvals includes a set of commonly used evaluators and is a great starting point if you’re new to evaluations. If you want greater flexibility in how you evaluate your apps, you can also define completely custom evaluators using your own code.\n\u200bLangSmith SDK\n\u200b1. Install dependencies\nPythonTypeScriptCopypip install -U langsmith openevals openai'

In [12]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [13]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001BEE7CD0ED0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001BEE7CD1BD0>, root_client=<openai.OpenAI object at 0x000001BEE7CD0A50>, root_async_client=<openai.AsyncOpenAI object at 0x000001BEE7CD18D0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, confi

In [15]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Evaluations are made up of three components:",
    "context":[Document(page_content="Evaluations are made up of three components:A dataset with test inputs and optionally expected outputs.A target function that defines what you’re evaluating. For example, this may be one LLM call that includes the new prompt you are testing, a part of your application or your end to end application.Evaluators that score your target functions outputs.")]
})

'Evaluations consist of a dataset with test inputs (and optionally expected outputs), a target function that specifies what is being evaluated (such as an LLM call with a new prompt, a part of an application, or an end-to-end application), and evaluators that score the outputs of the target function.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [16]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [17]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [18]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001BEE6FCB8D0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [19]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"Evaluations are made up of three components:"})
response['answer']

'Based on the provided context, one way to evaluate the performance of LLM applications is through the use of evaluations, which are quantitative methods designed to measure how well these models perform. This involves setting up environment variables, creating datasets, defining what you are evaluating, adding evaluators, and running the evaluations to view results. For instance, in the context provided, a dataset is created with specific input-output pairs (like questions and their correct answers), and an evaluator is added to assess correctness. The evaluation is then run to produce an experiment that analyzes the results, helping in identifying any failures and aiding in the development of more reliable AI applications.'

In [20]:

response

{'input': 'Evaluations are made up of three components:',
 'context': [Document(id='7f5c009f-d7ee-47dd-a31f-61ba18708222', metadata={'source': 'https://docs.langchain.com/langsmith/evaluation-quickstart', 'title': 'Evaluation quickstart - Docs by LangChain', 'language': 'en'}, page_content='Evaluations are made up of three components:'),
  Document(id='de2e94eb-261c-4915-84d5-7e539df4e286', metadata={'source': 'https://docs.langchain.com/langsmith/evaluation-quickstart', 'title': 'Evaluation quickstart - Docs by LangChain', 'language': 'en'}, page_content='up environment variables4. Create a dataset5. Define what you’re evaluating6. Define evaluator7. Run and view resultsNext stepsLangSmith UI1. Navigate to the playground2. Create a prompt3. Create a dataset4. Add an evaluator5. Run your evaluationNext stepsVideo guideQuickstartsEvaluation quickstartCopy pageCopy pageEvaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don’t alw

In [21]:
response['context']

[Document(id='7f5c009f-d7ee-47dd-a31f-61ba18708222', metadata={'source': 'https://docs.langchain.com/langsmith/evaluation-quickstart', 'title': 'Evaluation quickstart - Docs by LangChain', 'language': 'en'}, page_content='Evaluations are made up of three components:'),
 Document(id='de2e94eb-261c-4915-84d5-7e539df4e286', metadata={'source': 'https://docs.langchain.com/langsmith/evaluation-quickstart', 'title': 'Evaluation quickstart - Docs by LangChain', 'language': 'en'}, page_content='up environment variables4. Create a dataset5. Define what you’re evaluating6. Define evaluator7. Run and view resultsNext stepsLangSmith UI1. Navigate to the playground2. Create a prompt3. Create a dataset4. Add an evaluator5. Run your evaluationNext stepsVideo guideQuickstartsEvaluation quickstartCopy pageCopy pageEvaluations are a quantitative way to measure performance of LLM applications, which is important because LLMs don’t always behave predictably — small changes in prompts, models, or inputs ca